In [ ]:
pip install pyreadstat

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import pyreadstat as pyr
import json,csv

# matplotlib
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# scipy specifics
from scipy import stats as sps
from scipy.interpolate import interp1d

In [35]:
#data.to_csv('enaho_educacion_tercer.csv')
#data = pd.io.stata.read_stata('primer_trimestre_educ_2019.dta')
#data.to_csv('desercion_segundo.csv')
# data = pd.io.stata.read_stata('primer_trimestre_educ.dta')
#data = pd.read_spss('tercer_trimestre_educ.sav')
outputfile= 'json/data-iep.json'
data_name = "DB_noviembre.csv"
periodo = "Noviembre 2021"
dp = pd.read_csv(data_name)
pd.set_option('display.max_columns', None)
dp = dp.applymap(str)
dp

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,SbjNum,NSE,NSEv1,NSEv2,edad,edadx,DNI,sexo,dep,prov,dist,zona,region,ambito,edu,edu2,ocupa,ocupa2,p01,p01otro,p02,p03,p04,p05_1,p05_2,p05_3,p05_4,p06_1,p06_2,p06_3,p06_4,p06_5,p06_6,p06_7,p06_8,p06_9,p06_10,p07,p07otro,p08,p08otro,p11,p12,p12a,p13,p13a,p15_1,p15_2,p15_3,p15_4,PONDERADOR,PERCENTAGE,periodo
0,0,0,0,0,160518501,NSE C1,NSE C,NSE C,25,De 25 a 39 años,SI,Hombre,Puno,Puno,210101,Sur,2,Peru urbano,Superior universitaria completa,Educacion superior,Asalariado del sector privado,Trabajador dependiente,Corrupcion,,Desaprueba,Mejorará en los próximos meses,Durará cinco años como presidente,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Desaprueba,Aprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Aprueba,Fizcalizan bien a los ministros,,"No les interesa la gente, solo piensan en ellos",,Debe haber un retorno gradual de acuerdo con l...,No,Desaprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Algo probable,0.878,0.0725657616884184,NOVIEMBRE 2021
1,1,1,1,1,160519320,NSE C1,NSE C,NSE C,24,De 18 a 24 años,SI,Hombre,Lima,Lima,150135,Lima Metropolitana,1,Lima Metropolitana,Secundaria completa,Educacion Basica,Asalariado del sector privado,Trabajador dependiente,Economia,,Desaprueba,Empeorará en los próximos meses,No terminará su gobierno,Desaprueba,Desaprueba,Desaprueba,NS/NP,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,No tiene nada positivo,,La falta de gente capacitada,,Debe haber un retorno gradual de acuerdo con l...,No,Aprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Muy probable,1.2423,0.1026747673639205,NOVIEMBRE 2021
2,2,2,2,2,160519533,NSE B1,NSE B,NSE A/B,44,De 40 a más años,SI,Hombre,Lima,Lima,150128,Lima Metropolitana,1,Lima Metropolitana,Superior técnico incompleta,Educacion superior,Asalariado del sector privado,Trabajador dependiente,Inestabilidad política,NO DEJAN GOBERNAR AL PRESIDENTE,Aprueba,Mejorará en los próximos meses,Durará cinco años como presidente,Aprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,No tiene nada positivo,,No dejan gobernar al ejecutivo,,En marzo todos deberían regresar a clases pres...,No,Desaprueba,Centro,Centro,Poco probable,Muy probable,Algo probable,Nada probable,0.6874,0.0568128753811148,NOVIEMBRE 2021
3,3,3,3,3,160519780,NSE B2,NSE B,NSE A/B,36,De 25 a 39 años,SI,Hombre,Lima,Lima,150110,Lima Metropolitana,1,Lima Metropolitana,Superior técnico completa,Educacion superior,Trabajador independiente con empleados,Trabajador independiente,Economia,,Desaprueba,Empeorará en los próximos meses,No terminará su gobierno,Aprueba,Aprueba,Desaprueba,Desaprueba,NS/NP,NS/NP,Desaprueba,Desaprueba,Aprueba,NS/NP,Desaprueba,Desaprueba,Desaprueba,NS/NP,Fizcalizan bien a los ministros,,"No les interesa la gente, solo piensan en ellos",,Debe haber un retorno gradual de acuerdo con l...,Sí,Desaprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Muy probable,0.6388,0.0527961373195463,NOVIEMBRE 2021
4,4,4,4,4,160520180,NSE B2,NSE B,NSE A/B,18,De 18 a 24 años,SI,Hombre,Lima,Lima,150117,Lima Metropolitana,1,Lima Metropolitana,Secundaria completa,Educacion Basica,Asalariado del sector privado,Trabajador dependiente,Desigualdad,,Desaprueba,Se mantendrá igual los cinco años de su gobierno,No terminará su gobierno,Desaprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Desaprueba,Desaprueba,Desaprueba,Fizcalizan bien a los ministros,,No dejan gobernar al ejecutivo,,En marzo todos deberían regresar a clases pres...,Sí,Aprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Algo probable,0.6388,0.0527961373195463,NOVIEMBRE 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [3]:
def append2Json(file, key, dataframe, append=False):
    result = dataframe.to_json(orient="records")
    parsed = json.loads(result)
    data = {}
    with open(file, 'r+') as readfile:
        data = json.load(readfile)
        if(append):
            if(key in data):
                data[key] = data[key] + parsed 
            else: 
                data[key] = parsed
        else:
            data[key] = parsed
        readfile.close()
    with open(file, 'w') as writefile:
        json.dump(data,writefile)  

In [ ]:
dp["NSE"].replace({"1": "NSE A1", "2": "NSE A2", "3": "NSE B1", "4": "NSE B2", "5": "NSE C1", "6": "NSE C2", "7": "NSE D", "8": "NSE E"}, inplace=True)
dp["NSEv1"].replace({"1": "NSE A", "2": "NSE B", "3": "NSE C", "4": "NSE D", "5": "NSE E"}, inplace=True)
dp["NSEv2"].replace({"1": "NSE A/B", "2": "NSE C", "3": "NSE D/E"}, inplace=True)
dp["edadx"].replace({"1": "Menos de 18 años", "2": "De 18 a 24 años", "3": "De 25 a 39 años", "4": "De 40 a más años", "5": "NS/NP"}, inplace=True)
dp["DNI"].replace({"1": "SI", "2": "NO", "3": "NS/NP"}, inplace=True)
dp["sexo"].replace({"1": "Hombre", "2": "Mujer","3": "Otro"}, inplace=True)
dp["zona"].replace({"1": "Lima Metropolitana", "2": "Norte", "3": "Centro", "4": "Sur", "5": "Selva"}, inplace=True)
#dp["edu"].replace({"1": "Sin educacion/Educacion Inicial", "2": "Primaria incompleta o completa/Secundaria incompleta", "3": "Secundaria completa", "4": "Superior técnico incompleta", "5": "Superior técnico completa", "6": "Superior universitaria incompleta", "7": "Superior universitaria completa", "8": "Posgrado universitario"}, inplace=True)
#dp["edu2"].replace({"1": "Educacion Basica", "2": "Educacion superior"}, inplace=True)
dp["ambito"].replace({"1": "Lima Metropolitana", "2": "Perú urbano", "3": "Perú rural", "4": "NS/NP"})
#dp["ocupa"].replace({"1": "Asalariado del sector estatal", "2": "Asalariado del sector privado", "3": "Trabajador independiente con empleados", "4": "Trabajador independiente sin empleados", "5": "Agricultor", "6": "Empleado del hogar", "7": "Estudiante", "8":"Jubilado", "9": "Ama de casa", "10": "No trabaja"}, inplace=True)
#dp["ocupa2"].replace({"1": "Trabajador dependiente", "2": "Trabajador independiente", "3": "No trabaja"}, inplace=True)
dp["p01"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_1"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_2"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_3"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_4"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_5"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_6"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p02_7"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
dp["p03_1"].replace({"1": "Nada", "2": "Poco", "3": "Algo", "4": "Mucho", "99":"NS/NP"}, inplace=True)
dp["p03_2"].replace({"1": "Nada", "2": "Poco", "3": "Algo", "4": "Mucho", "99":"NS/NP"}, inplace=True)
dp["p03_3"].replace({"1": "Nada", "2": "Poco", "3": "Algo", "4": "Mucho", "99":"NS/NP"}, inplace=True)
dp["p03_4"].replace({"1": "Nada", "2": "Poco", "3": "Algo", "4": "Mucho", "99":"NS/NP"}, inplace=True)
dp["p04_1"].replace({"1": "Muy mal", "2": "Mal", "3": "Regular", "4":"Bien", "5":"Muy bien","99":"NS/NP"}, inplace=True)
dp["p04_2"].replace({"1": "Muy mal", "2": "Mal", "3": "Regular", "4":"Bien", "5":"Muy bien","99":"NS/NP"}, inplace=True)
dp["p04_3"].replace({"1": "Muy mal", "2": "Mal", "3": "Regular", "4":"Bien", "5":"Muy bien","99":"NS/NP"}, inplace=True)
dp["p04_4"].replace({"1": "Muy mal", "2": "Mal", "3": "Regular", "4":"Bien", "5":"Muy bien","99":"NS/NP"}, inplace=True)
dp["p04_5"].replace({"1": "Muy mal", "2": "Mal", "3": "Regular", "4":"Bien", "5":"Muy bien","99":"NS/NP"}, inplace=True)
#dp["p05"].replace({"1": "Que se mantenga todo el gabinete", "2": "Que se cambie a algunos ministros","3": "Que se cambie todo el gabinete", "4": "NS/NP"}, inplace=True)
#dp["p06"].replace({"1": "Miedo", "2": "Esperanza","3": "Incertidumbre", "4": "Confianza", "5": "Orgullo", "6":"Desilusión", "7":"Desconfianza", "8":"Otro", "99":"NS/NP"}, inplace=True)
dp["p07"].replace({"1": "Muy mal", "2": "Mal", "3": "Regular", "4":"Bien", "5":"Muy bien","99":"NS/NP"}, inplace=True)
dp["p08"].replace({"1": "Si", "2": "No","99":"NS/NP"}, inplace=True)
dp["p09"].replace({"1": "Si", "2": "No","99":"NS/NP"}, inplace=True)
#dp["p11"].replace({"1": "Atenta contra la libertad de prensa", "2": "No atenta contra la libertad de prensa","3": "NS/NP"}, inplace=True)
#dp["p12"].replace({"1": "Izquierda", "2": "Izquierda","3": "Izquierda", "4": "Izquierda", "5":"Centro", "6":"Centro", "7":"Derecha", "8":"Derecha", "9": "Derecha", "10":"Derecha","99":"NS/NP"}, inplace=True)

In [ ]:
#dp["p05_4"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
#dp["p01"].replace({"1": "Aprueba", "2": "Desaprueba", "99":"NS/NP"}, inplace=True)
#dp["Identificacion_ideológica"].replace({"1": "Izquierda", "2": "Centro","3": "Derecha","Sistema":"NS/NP"}, inplace=True)
dp["ambito"].replace({"1": "Lima Metropolitana", "Perú urbano": "Peru urbano", "Perú rural": "Peru rural", "4": "NS/NP"}, inplace=True)

In [36]:
#np.finfo(np.float64)
pd.set_option('display.float_format', '{:.2f}'.format)
dp = dp.rename({'pondera': 'PONDERADOR'}, axis=1)  # renombrar columna

In [37]:
#suma de valores nulos
pd.options.display.max_rows = 999
dp.isna().sum()

Unnamed: 0          0
Unnamed: 0.1        0
Unnamed: 0.1.1      0
Unnamed: 0.1.1.1    0
SbjNum              0
NSE                 0
NSEv1               0
NSEv2               0
edad                0
edadx               0
DNI                 0
sexo                0
dep                 0
prov                0
dist                0
zona                0
region              0
ambito              0
edu                 0
edu2                0
ocupa               0
ocupa2              0
p01                 0
p01otro             0
p02                 0
p03                 0
p04                 0
p05_1               0
p05_2               0
p05_3               0
p05_4               0
p06_1               0
p06_2               0
p06_3               0
p06_4               0
p06_5               0
p06_6               0
p06_7               0
p06_8               0
p06_9               0
p06_10              0
p07                 0
p07otro             0
p08                 0
p08otro             0
p11       

In [38]:
#Poblacion total
dp['PONDERADOR'] = dp['PONDERADOR'].astype(float)
suma_factor_total = dp['PONDERADOR'].sum(axis=0)

suma_factor_total

1209.9370000000001

In [ ]:
df_dict = pd.read_csv('dpto.csv', index_col=False)
df_dict

In [ ]:
df_dict.astype({'codigo': 'str'}).dtypes
df_dict['codigo'] = df_dict['codigo'].astype(str)

In [ ]:
ubigeo_dict = df_dict.to_dict('split')
ubigeo_dict['data'][0][1]

In [ ]:
arrays =[]
for x in range(len(ubigeo_dict['data'])):
    array = ubigeo_dict['data'][x][0]
    arrays.append(array)
    
arrays

In [ ]:
dic = {}
for i, l in enumerate(ubigeo_dict['data']):    
    dic = dict(zip(arrays, list(dic.values()))) 
    dic[i] = l[1]
    
dic

In [ ]:
dp['prov'] = dp['prov'].map(dic)

In [39]:
dp["PERCENTAGE"] = (dp["PONDERADOR"] / dp['PONDERADOR'].sum())*100

In [40]:
dp['periodo'] = periodo
dp['periodo'] = dp['periodo'].str.upper()
dp

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,SbjNum,NSE,NSEv1,NSEv2,edad,edadx,DNI,sexo,dep,prov,dist,zona,region,ambito,edu,edu2,ocupa,ocupa2,p01,p01otro,p02,p03,p04,p05_1,p05_2,p05_3,p05_4,p06_1,p06_2,p06_3,p06_4,p06_5,p06_6,p06_7,p06_8,p06_9,p06_10,p07,p07otro,p08,p08otro,p11,p12,p12a,p13,p13a,p15_1,p15_2,p15_3,p15_4,PONDERADOR,PERCENTAGE,periodo
0,0,0,0,0,160518501,NSE C1,NSE C,NSE C,25,De 25 a 39 años,SI,Hombre,Puno,Puno,210101,Sur,2,Peru urbano,Superior universitaria completa,Educacion superior,Asalariado del sector privado,Trabajador dependiente,Corrupcion,,Desaprueba,Mejorará en los próximos meses,Durará cinco años como presidente,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Desaprueba,Aprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Aprueba,Fizcalizan bien a los ministros,,"No les interesa la gente, solo piensan en ellos",,Debe haber un retorno gradual de acuerdo con l...,No,Desaprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Algo probable,0.88,0.07,NOVIEMBRE 2021
1,1,1,1,1,160519320,NSE C1,NSE C,NSE C,24,De 18 a 24 años,SI,Hombre,Lima,Lima,150135,Lima Metropolitana,1,Lima Metropolitana,Secundaria completa,Educacion Basica,Asalariado del sector privado,Trabajador dependiente,Economia,,Desaprueba,Empeorará en los próximos meses,No terminará su gobierno,Desaprueba,Desaprueba,Desaprueba,NS/NP,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,No tiene nada positivo,,La falta de gente capacitada,,Debe haber un retorno gradual de acuerdo con l...,No,Aprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Muy probable,1.24,0.10,NOVIEMBRE 2021
2,2,2,2,2,160519533,NSE B1,NSE B,NSE A/B,44,De 40 a más años,SI,Hombre,Lima,Lima,150128,Lima Metropolitana,1,Lima Metropolitana,Superior técnico incompleta,Educacion superior,Asalariado del sector privado,Trabajador dependiente,Inestabilidad política,NO DEJAN GOBERNAR AL PRESIDENTE,Aprueba,Mejorará en los próximos meses,Durará cinco años como presidente,Aprueba,Desaprueba,Desaprueba,Desaprueba,Desaprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,No tiene nada positivo,,No dejan gobernar al ejecutivo,,En marzo todos deberían regresar a clases pres...,No,Desaprueba,Centro,Centro,Poco probable,Muy probable,Algo probable,Nada probable,0.69,0.06,NOVIEMBRE 2021
3,3,3,3,3,160519780,NSE B2,NSE B,NSE A/B,36,De 25 a 39 años,SI,Hombre,Lima,Lima,150110,Lima Metropolitana,1,Lima Metropolitana,Superior técnico completa,Educacion superior,Trabajador independiente con empleados,Trabajador independiente,Economia,,Desaprueba,Empeorará en los próximos meses,No terminará su gobierno,Aprueba,Aprueba,Desaprueba,Desaprueba,NS/NP,NS/NP,Desaprueba,Desaprueba,Aprueba,NS/NP,Desaprueba,Desaprueba,Desaprueba,NS/NP,Fizcalizan bien a los ministros,,"No les interesa la gente, solo piensan en ellos",,Debe haber un retorno gradual de acuerdo con l...,Sí,Desaprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Muy probable,0.64,0.05,NOVIEMBRE 2021
4,4,4,4,4,160520180,NSE B2,NSE B,NSE A/B,18,De 18 a 24 años,SI,Hombre,Lima,Lima,150117,Lima Metropolitana,1,Lima Metropolitana,Secundaria completa,Educacion Basica,Asalariado del sector privado,Trabajador dependiente,Desigualdad,,Desaprueba,Se mantendrá igual los cinco años de su gobierno,No terminará su gobierno,Desaprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Aprueba,Desaprueba,Desaprueba,Aprueba,Aprueba,Desaprueba,Desaprueba,Desaprueba,Fizcalizan bien a los ministros,,No dejan gobernar al ejecutivo,,En marzo todos deberían regresar a clases pres...,Sí,Aprueba,Centro,Centro,Muy probable,Muy probable,Muy probable,Algo probable,0.64,0.05,NOVIEMBRE 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,1205,1205,1205,1205,160734889,NSE

In [ ]:
dp.to_csv(data_name)

In [ ]:
#P01. ¿Usted aprueba o desaprueba la forma como PEDRO CASTILLO está conduciendo su gobierno? (agosto, setiembre, octubre)
#P02. ¿Usted aprueba o desaprueba la forma como PEDRO CASTILLO está conduciendo su gobierno? (noviembre)

aprobacion_PC = dp.groupby(["p02", "periodo"]).agg({'PERCENTAGE': 'sum'})
aprobacion_PC = aprobacion_PC.reset_index()
aprobacion_PC

In [ ]:
append2Json(outputfile, "aprobacion_noviembre", aprobacion_PC)

In [ ]:
#P02.2 ¿Usted aprueba o desaprueba el desempeño de...? - El Congreso de la República (Agosto)
#P05.3 ¿Usted aprueba o desaprueba el desempeño de...? - El Congreso de la República (Noviembre)
#P02.3 ¿Usted aprueba o desaprueba el desempeño de...? - El Congreso de la República (Setiembre)
#P05.1 ¿Usted aprueba o desaprueba el desempeño de...? - El Congreso de la República (Octubre)

aprobacion_congreso = dp.groupby(["p05_3", "periodo"]).agg({'PERCENTAGE': 'sum'})
aprobacion_congreso = aprobacion_congreso.reset_index()
aprobacion_congreso

In [ ]:
append2Json(outputfile, "congreso_octubre", aprobacion_congreso)

In [43]:
#P01. ¿Usted aprueba o desaprueba la forma como PEDRO CASTILLO está conduciendo su gobierno? (agosto, setiembre, octubre)
#P02. ¿Usted aprueba o desaprueba la forma como PEDRO CASTILLO está conduciendo su gobierno? (noviembre)

# aprobacion_nse = dp.groupby(["p01", "periodo", "NSEv1"]).agg({'PERCENTAGE': 'sum'})
# aprobacion_nse = aprobacion_nse.reset_index()
# aprobacion_nse

percentage_nse = pd.DataFrame()
for response in ['NSE A/B', 'NSE C', 'NSE D/E']:
    db_zona = dp[dp["NSEv2"] == response]
    this_aprobacion = db_zona['p02'] == "Aprueba"
    data_aprueba = db_zona[this_aprobacion]   
    #data_aprueba["aprobacion"] = (data_aprueba["PONDERADOR"] / data_aprueba['PONDERADOR'].sum())*100
    data_aprueba["periodo"] = periodo
    data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100
    percentage_nse = percentage_nse.append(data_aprueba)
    
    percentage_nse['zona'] = percentage_nse['zona'].str.upper()
    percentage_nse['prov'] = percentage_nse['prov'].str.upper()
    percentage_nse['periodo'] = percentage_nse['periodo'].str.upper()
    
    
#percentage_zona.groupby(["zona", "periodo", "aprobacion"])

result = percentage_nse[['NSEv2', 'aprobacion', 'periodo']]
result = result.drop_duplicates()
result

<ipython-input-43-ef5deaa2451c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aprueba["periodo"] = periodo
<ipython-input-43-ef5deaa2451c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100


,NSEv2,aprobacion,periodo
2,NSE A/B,13.13,NOVIEMBRE 2021
6,NSE C,17.72,NOVIEMBRE 2021
7,NSE D/E,35.48,NOVIEMBRE 2021


In [ ]:
append2Json(outputfile, "nse_setiembre", aprobacion_nse)

In [9]:

# provincias_zona = dp.groupby(["zona", "p02"]).agg({'PERCENTAGE': 'sum'})
# provincias_zona["PERCENTAGE_ZONAS"] = (dp["PONDERADOR"] / dp['PONDERADOR'].sum())*100
# provincias_zona = provincias_zona.reset_index()
# provincias_zona


percentage_zona = pd.DataFrame()
for response in ['Centro', 'Lima Metropolitana', 'Selva', 'Norte', 'Sur']:
    db_zona = dp[dp["zona"] == response]
    this_aprobacion = db_zona['p02'] == "Desaprueba"
    data_aprueba = db_zona[this_aprobacion]   
    #data_aprueba["aprobacion"] = (data_aprueba["PONDERADOR"] / data_aprueba['PONDERADOR'].sum())*100
    data_aprueba["periodo"] = periodo
    data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100
    percentage_zona = percentage_zona.append(data_aprueba)
    
    percentage_zona['zona'] = percentage_zona['zona'].str.upper()
    percentage_zona['prov'] = percentage_zona['prov'].str.upper()
    percentage_zona['periodo'] = percentage_zona['periodo'].str.upper()
    
    
#percentage_zona.groupby(["zona", "periodo", "aprobacion"])

result = percentage_zona[['zona', 'prov', 'aprobacion', 'periodo']]
result = result.drop_duplicates()
result

<ipython-input-9-1025ab159908>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aprueba["periodo"] = periodo
<ipython-input-9-1025ab159908>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100


,zona,prov,aprobacion,periodo
45,CENTRO,CAÑETE,60.96,NOVIEMBRE 2021
52,CENTRO,TARMA,60.96,NOVIEMBRE 2021
64,CENTRO,HUARAL,60.96,NOVIEMBRE 2021
149,CENTRO,HUANCAYO,60.96,NOVIEMBRE 2021
178,CENTRO,BARRANCA,60.96,NOVIEMBRE 2021
238,CENTRO,HUAURA,60.96,NOVIEMBRE 2021
321,CENTRO,DANIEL ALCIDES CARRIÓN,60.96,NOVIEMBRE 2021
370,CENTRO,HUÁNUCO,60.96,NOVIEMBRE 2021
469,CENTRO,PASCO,60.96,NOVIEMBRE 2021
470,CENTRO,HUAROCHIRÍ,60.96,NOVIEMBRE 2021


In [ ]:
result.to_json('json/aprobacion_macrozonas_agosto.json', orient="table", index=False)
#append2Json(outputfile, "desercion_nivel_educ", matriculadosDF)

In [ ]:
#D08A. Ámbito

# aprobacion_ambito = dp.groupby(["p01", "ambito", "periodo"]).agg({'PERCENTAGE': 'sum'})
# aprobacion_ambito = aprobacion_ambito.reset_index()
# aprobacion_ambito

percentage_ambito = pd.DataFrame()
for response in ['Lima Metropolitana', 'Peru rural', 'Peru urbano']:
    db_zona = dp[dp["ambito"] == response]
    this_aprobacion = db_zona['p02'] == "Aprueba"
    data_aprueba = db_zona[this_aprobacion]   
    #data_aprueba["aprobacion"] = (data_aprueba["PONDERADOR"] / data_aprueba['PONDERADOR'].sum())*100
    data_aprueba["periodo"] = periodo
    data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100
    percentage_ambito = percentage_ambito.append(data_aprueba)
    
    percentage_ambito['zona'] = percentage_ambito['zona'].str.upper()
    percentage_ambito['prov'] = percentage_ambito['prov'].str.upper()
    percentage_ambito['periodo'] = percentage_ambito['periodo'].str.upper()
    
    
#percentage_zona.groupby(["zona", "periodo", "aprobacion"])

result = percentage_ambito[['ambito', 'aprobacion', 'periodo']]
result = result.drop_duplicates()
result

In [ ]:
append2Json(outputfile, "ambito_diciembre", aprobacion_ambito)

In [ ]:
#P04. ¿Cree que Pedro Castillo…?
# cree_castillo = dp.groupby(["p04", "periodo"]).agg({'PERCENTAGE': 'sum'})
# cree_castillo = cree_castillo.reset_index()
# cree_castillo

percentage_dudas = pd.DataFrame()
for response in ['Durará cinco años como presidente', 'No terminará su gobierno']:
    db_zona = dp[dp["p04"] == response]
    this_aprobacion = db_zona['p02'] == "Desaprueba"
    data_aprueba = db_zona[this_aprobacion]   
    #data_aprueba["aprobacion"] = (data_aprueba["PONDERADOR"] / data_aprueba['PONDERADOR'].sum())*100
    data_aprueba["periodo"] = periodo
    data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100
    percentage_dudas = percentage_dudas.append(data_aprueba)
    
    percentage_dudas['zona'] = percentage_dudas['zona'].str.upper()
    percentage_dudas['prov'] = percentage_dudas['prov'].str.upper()
    percentage_dudas['periodo'] = percentage_dudas['periodo'].str.upper()
    
    
#percentage_zona.groupby(["zona", "periodo", "aprobacion"])

result = percentage_dudas[['p04', 'aprobacion', 'periodo']]
result = result.drop_duplicates()
result

In [10]:
percentage_dudas = dp.groupby(["p04", "periodo"]).agg({'PERCENTAGE': 'sum'})
percentage_dudas = percentage_dudas.reset_index()
percentage_dudas

,p04,periodo,PERCENTAGE
0,Durará cinco años como presidente,NOVIEMBRE 2021,30.58
1,NS/NP,NOVIEMBRE 2021,7.00
2,No terminará su gobierno,NOVIEMBRE 2021,62.42


In [ ]:
append2Json(outputfile, "creeqcastillo_diciembre", cree_castillo)

In [44]:
#P05.4 ¿Usted aprueba o desaprueba el desempeño de...? - La presidenta del Congreso, María del Carmen Alva (noviembre)
#P02.3 ¿Usted aprueba o desaprueba el desempeño de...? - La presidenta del Congreso, María del Carmen Alva (agosto)
#P02.4 ¿Usted aprueba o desaprueba el desempeño de...? - La presidenta del Congreso, María del Carmen Alva (setiembre)
#P05.2 ¿Usted aprueba o desaprueba el desempeño de...? - La presidenta del Congreso, María del Carmen Alva (octubre)

aprobacion_MCA = dp.groupby(["p05_4", "periodo"]).agg({'PERCENTAGE': 'sum'})
aprobacion_MCA = aprobacion_MCA.reset_index()
aprobacion_MCA

,p05_4,periodo,PERCENTAGE
0,Aprueba,NOVIEMBRE 2021,30.68
1,Desaprueba,NOVIEMBRE 2021,61.10
2,NS/NP,NOVIEMBRE 2021,8.21


In [ ]:
append2Json(outputfile, "aprobacion_MCA_diciembre", aprobacion_MCA)

In [ ]:
#P12. Tenemos una escala del 1 al 10 que va de izquierda a derecha, en la que el número 1 significa “izquierda” y el 10 significa “derecha”. Cuando piensa sobre su punto de vista político, ¿dónde se encontraría usted en esta escala? (agosto, setiembre)
#Identificación ideológica (octubre)
#P13. Tenemos una escala del 1 al 10 que va de izquierda a derecha, en la que el número 1 significa “izquierda” y el 10 significa “derecha”. Cuando piensa sobre su punto de vista político, ¿dónde se encontraría usted en esta escala? Dígame un


# aprobacion_ideologia = dp.groupby(["p01","Identificacion_ideológica","periodo"]).agg({'PERCENTAGE': 'sum'})
# aprobacion_ideologia = aprobacion_ideologia.reset_index()
# aprobacion_ideologia

percentage_ideologia = pd.DataFrame()
for response in ['Izquierda', 'Centro', 'Derecha']:
    db_zona = dp[dp["Identificacion_ideológica"] == response]
    this_aprobacion = db_zona['p01'] == "Desaprueba"
    data_aprueba = db_zona[this_aprobacion]   
    #data_aprueba["aprobacion"] = (data_aprueba["PONDERADOR"] / data_aprueba['PONDERADOR'].sum())*100
    data_aprueba["periodo"] = periodo
    data_aprueba["aprobacion"] = (data_aprueba['PERCENTAGE'].sum() / db_zona['PERCENTAGE'].sum())*100
    percentage_ideologia = percentage_ideologia.append(data_aprueba)
    
    percentage_ideologia['zona'] = percentage_ideologia['zona'].str.upper()
    percentage_ideologia['prov'] = percentage_ideologia['prov'].str.upper()
    percentage_ideologia['periodo'] = percentage_ideologia['periodo'].str.upper()
    
    
#percentage_zona.groupby(["zona", "periodo", "aprobacion"])

result = percentage_ideologia[['Identificacion_ideológica', 'aprobacion', 'periodo']]
result = result.drop_duplicates()
result

In [ ]:
append2Json(outputfile, "aprobacion_ideologia_octubre", result)

In [ ]:
provincia = dp.groupby(["dep"]).agg({'PONDERADOR': 'sum'})
provincia = provincia.reset_index()

provincia

In [ ]:
#razones nan
#dp['P313'] = dp['P313'].fillna('no respondió')

In [ ]:
this_provincia = dp['prov'] == "Purús"
data_provincia = dp[this_provincia]
data_provincia

In [ ]:
#P12A. ¿Usted aprueba o desaprueba que el Congreso pida la vacancia del presidente Pedro Castillo?
#P13a. Identificación ideológica

preguntas_rpta = dp.groupby(["p13a", "p12a"]).agg({'PONDERADOR': 'sum'})
preguntas_rpta = preguntas_rpta.reset_index()
preguntas_rpta

In [ ]:
#D10. ¿Cuál es su ocupación principal?
#P01. En su opinión, ¿cuál es el principal problema del país?

preguntas_rpta = dp.groupby(["p01","ocupa"]).size().unstack()
preguntas_rpta = preguntas_rpta.reset_index()
preguntas_rpta

In [ ]:
#P04. ¿Cree que Pedro Castillo…?
#D03. ¿En qué departamento del Perú vive usted?
#p13a = identificacion ideologica

preguntas_rpta = dp.groupby(["dep", "p04"]).size().unstack()
preguntas_rpta = preguntas_rpta.reset_index()
preguntas_rpta

In [ ]:
#P04. ¿Cree que Pedro Castillo…?
#D03. ¿En qué departamento del Perú vive usted?
#p12a = ¿Usted aprueba o desaprueba que el Congreso pida la vacancia del presidente Pedro Castillo?


preguntas_rpta = dp.groupby(["dep", "p12a"]).size().unstack()
preguntas_rpta = preguntas_rpta.reset_index()
preguntas_rpta

In [ ]:
# P06.1 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Acción Popular
# P06.2 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Alianza para el Progreso
# P06.3 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Avanza País
# P06.4 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Fuerza Popular
# P06.5 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Juntos por el Perú
# P06.6 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Partido Morado
# P06.7 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Perú Libre
# P06.8 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Podemos Perú
# P06.9 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Renovación Popular
# P06.10 ¿Usted aprueba o desaprueba la forma en la que está actuando la bancada de … en el Congreso? - Somos Perú


preguntas_rpta = dp.groupby(["p06_6"]).sum()
preguntas_rpta = preguntas_rpta.reset_index()
preguntas_rpta

In [ ]:
#P07. En su opinión ¿qué es lo mejor del actual Congreso?
#dep. ¿En qué departamento del Perú vive usted?
#p13a = identificacion ideologica

preguntas_rpta = dp.groupby(["dep", "p07"]).size().unstack()
preguntas_rpta = preguntas_rpta.reset_index()
preguntas_rpta

In [ ]:
matriculadosDF = pd.DataFrame()
for response in ['si', 'no']:
    matriculados_2020 = educacion_basica_incompleta['P306'] == response
#     centro_matriculados = data_menor_25['P308D'].notna()
    data_matriculados_2020 = educacion_basica_incompleta[matriculados_2020]
# matriculados_basica = data_matriculados_2020.groupby(["P301A"]).size()
    matriculados_basica = data_matriculados_2020.groupby(["P301A", "P306"]).agg({'FACTOR': 'sum'})
    matriculados_basica = matriculados_basica.reset_index()
    matriculados_basica["PERCENTAGE"] = (matriculados_basica["FACTOR"] / matriculados_basica['FACTOR'].sum())*100
    matriculados_basica["MATRICULADOS"] = np.where(matriculados_basica["P306"] == "si","Matriculados", "No matriculados")
    matriculados_basica["TRIMESTRE"] = trimestre
    matriculadosDF = matriculadosDF.append(matriculados_basica)

matriculadosDF

In [ ]:
append2Json(outputfile, "desercion_nivel_educ", matriculadosDF)

In [ ]:
matriculadosDF_nivel = pd.DataFrame()
for response in ['básica especial', 'educación inicial', 'primaria completa', 'primaria incompleta', 'secundaria incompleta']:
    matriculados_2020 = educacion_basica_incompleta['P301A'] == response
#     centro_matriculados = data_menor_25['P308D'].notna()
    data_matriculados_2020 = educacion_basica_incompleta[matriculados_2020]
# matriculados_basica = data_matriculados_2020.groupby(["P301A"]).size()
    matriculados_basica = data_matriculados_2020.groupby(["P301A", "P306"]).agg({'FACTOR': 'sum'})
    matriculados_basica = matriculados_basica.reset_index()
    matriculados_basica["PERCENTAGE"] = (matriculados_basica["FACTOR"] / matriculados_basica['FACTOR'].sum())*100
    matriculados_basica["MATRICULADOS"] = np.where(matriculados_basica["P306"] == "si","Matriculados", "No matriculados")
    matriculados_basica["TRIMESTRE"] = trimestre
    matriculadosDF_nivel = matriculadosDF_nivel.append(matriculados_basica)

matriculadosDF_nivel

In [ ]:
no_matriculados_2020 = educacion_basica_incompleta['P306'] == 'no'
data_no_matriculados = educacion_basica_incompleta[no_matriculados_2020]
años_nivel = data_no_matriculados.groupby(["P301A", "P208A"]).agg({'FACTOR': 'sum'})
años_nivel

In [ ]:
edadDf = pd.DataFrame()
for edad in range(6, 25):
    matriculados_2020 = educacion_basica_incompleta['P208A'] == edad
    centro_matriculados = educacion_basica_incompleta['P301A'].notna()
    data_matriculados_2020 = educacion_basica_incompleta[matriculados_2020 & centro_matriculados]
    # matriculados_basica = data_matriculados_2020.groupby(["P301A"]).size()
    matriculados_basica_edad = data_matriculados_2020.groupby(["P306"]).agg({'FACTOR': 'sum'})
    matriculados_basica_edad["PERCENTAGE"] = (matriculados_basica_edad["FACTOR"] / matriculados_basica_edad['FACTOR'].sum())*100
    matriculados_basica_edad["AÑOS"] = str(edad) + " AÑOS"
    matriculados_basica_edad["TRIMESTRE"] = trimestre
    matriculados_basica_edad = matriculados_basica_edad.reset_index()
    matriculados_basica_edad["MATRICULADOS"] = np.where(matriculados_basica_edad["P306"] == "si","Matriculados", "No matriculados")
    edadDf = edadDf.append(matriculados_basica_edad, ignore_index=True)
    
edadDf

In [ ]:
append2Json(outputfile, "chart_matriculados_edad", edadDf)

In [ ]:
#centro de estudios donde termino el ultimo nivel aprobado y en el que está matriculado actualmente
matriculados_2020 = educacion_basica_incompleta['P306'] == 'si'
data_matriculados_2020 = educacion_basica_incompleta[matriculados_2020]
# matriculados_estudios = data_matriculados_2020.groupby(["P301D","P308D"]).size()
matriculados_estudios = data_matriculados_2020.groupby(["P301D", "P308D", "P306"]).agg({'FACTOR': 'sum'})
matriculados_estudios["PERCENTAGE"] = (matriculados_estudios["FACTOR"] / matriculados_estudios['FACTOR'].sum())*100
matriculados_estudios = matriculados_estudios.reset_index()
matriculados_estudios["MATRICULADOS"] = np.where(matriculados_estudios["P306"] == "si","Matriculados", "No matriculados")
matriculados_estudios["TRIMESTRE"] = trimestre
matriculados_estudios["CATEGORIA"] = "De " + matriculados_estudios["P301D"].astype(str) + " a " + matriculados_estudios["P308D"]
matriculados_estudios


In [ ]:
#append2Json("json/data-2020-01.json", "data_nivel_centro", matriculados_estudios, True)

In [ ]:
# ULTIMO NIVEL POR LENGUA MATERNA
# data_matriculados_2020 = educacion_basica_incompleta[no_matriculados_2020]
lengua_educ_basica = educacion_basica_incompleta.groupby(["P301A", "P300A"]).agg({'FACTOR': 'sum'})
lengua_educ_basica = lengua_educ_basica.reset_index()
lengua_educ_basica

In [ ]:
#Cuantas personas q no han terminado la educ. básica hasta los 25 años no se han matriculado este año X lengua

matriculados_2020 = educacion_basica_incompleta['P306'] == 'no'
data_matriculados_2020 = educacion_basica_incompleta[matriculados_2020]
data_matriculados_edad_25 = data_matriculados_2020.groupby(['P301A', 'P300A']).agg({'FACTOR': 'sum'})
data_matriculados_edad_25 = data_matriculados_edad_25.reset_index()
data_matriculados_edad_25

In [ ]:
#porcentajes de no matriculados y total por lengua
comparativa_lengua_nomatriculados = data_matriculados_edad_25.merge(lengua_educ_basica, on=['P301A','P300A'], suffixes=['_df1','_df2']).eval('PERCENTAGE=(FACTOR_df1/FACTOR_df2)*100')
comparativa_lengua_nomatriculados['PERCENTAGE'] = comparativa_lengua_nomatriculados['PERCENTAGE'].round(2)
comparativa_lengua_nomatriculados["TRIMESTRE"] = trimestre
comparativa_lengua_nomatriculados

In [ ]:
append2Json(outputfile, "chart_bubbles_lengua", comparativa_lengua_nomatriculados)

In [ ]:
dominio = educacion_basica_incompleta.groupby(["P301A"]).agg({'FACTOR': 'sum'})
dominio = dominio.reset_index()
dominio

In [ ]:
data_dominio = educacion_basica_incompleta['P306'] == 'no'
no_matriculados_dominio = educacion_basica_incompleta[data_dominio]
data_nomatriculados_dominio = no_matriculados_dominio.groupby(['P301A', 'DOMINIO']).agg({'FACTOR': 'sum'})
data_nomatriculados_dominio = data_nomatriculados_dominio.reset_index()

data_nomatriculados_dominio

In [ ]:
comparativa_nomatriculados_dominio = data_nomatriculados_dominio.merge(dominio, on=['P301A'], suffixes=['_df1','_df2']).eval('PERCENTAGE=(FACTOR_df1/FACTOR_df2)*100')
comparativa_nomatriculados_dominio['PERCENTAGE'] = comparativa_nomatriculados_dominio['PERCENTAGE'].round(2)
comparativa_nomatriculados_dominio["TRIMESTRE"] = trimestre
comparativa_nomatriculados_dominio

In [ ]:
#Matriculados vs estudiando
matriculados_vs_estudiantes = educacion_basica_incompleta.groupby(["P306", "P307"]).agg({'FACTOR': 'sum'})
matriculados_vs_estudiantes = matriculados_vs_estudiantes.apply(np.round, decimals=2).reset_index()
matriculados_vs_estudiantes

In [ ]:
niveles_matriculados = educacion_basica_incompleta.groupby("P308A").agg({'FACTOR': 'sum'})
niveles = list(niveles_matriculados.index)
niveles

In [ ]:
medioDf = pd.DataFrame()
        
for nivel in niveles:
    matriculado_estudiando = educacion_basica_incompleta['P307'] == 'si'
    matriculado_estudiando_nivel = educacion_basica_incompleta['P308A'] == nivel
    data_matriculado_estudiando = educacion_basica_incompleta[matriculado_estudiando & matriculado_estudiando_nivel]
    group_matriculado_estudiando = data_matriculado_estudiando.groupby(["P308A", "P307A1", "P307A2", "P307A3", "P307A4"]).agg({'FACTOR': 'sum'})
    group_matriculado_estudiando = group_matriculado_estudiando.reset_index()
    group_matriculado_estudiando['PERCENTAGE'] = (group_matriculado_estudiando["FACTOR"] / group_matriculado_estudiando['FACTOR'].sum())*100
    group_matriculado_estudiando['PERCENTAGE'] = group_matriculado_estudiando['PERCENTAGE'].round(2)
    group_matriculado_estudiando["TRIMESTRE"] = trimestre 
    group_matriculado_estudiando.loc[group_matriculado_estudiando['P307A1'] == "pase", 'P307A1'] = np.nan
    group_matriculado_estudiando.loc[group_matriculado_estudiando['P307A2'] == "pase", 'P307A2'] = np.nan
    group_matriculado_estudiando.loc[group_matriculado_estudiando['P307A3'] == "pase", 'P307A3'] = np.nan
    group_matriculado_estudiando.loc[group_matriculado_estudiando['P307A4'] == "pase", 'P307A4'] = np.nan
    group_matriculado_estudiando['P307A1'] = group_matriculado_estudiando['P307A1'].replace({"televisión": "TV"})
    group_matriculado_estudiando['P307A2'] = group_matriculado_estudiando['P307A2'].replace({"radio / equipo de sonido": "Radio"})
    group_matriculado_estudiando['P307A3'] = group_matriculado_estudiando['P307A3'].replace({"plataforma virtual / página web": "Web"})
    group_matriculado_estudiando['P307A4'] = group_matriculado_estudiando['P307A4'].replace({"otro": "Otro"})
    group_matriculado_estudiando["CATEGORIA"] = group_matriculado_estudiando[['P307A1', 'P307A2', 'P307A3', 'P307A4']].stack().groupby(level=0).agg(' + '.join)
    group_matriculado_estudiando["CATEGORIA"] = group_matriculado_estudiando['CATEGORIA'].astype(str).replace({"nan": "Otro"})
    aggregation_functions = {'P308A':'first', 'P307A1':'first', 'P307A2':'first', 'P307A3':'first', 'P307A4':'first', 'FACTOR':'sum','PERCENTAGE': 'sum', 'TRIMESTRE':'first','CATEGORIA': 'first'}
    group_matriculado_estudiando = group_matriculado_estudiando.groupby(group_matriculado_estudiando['CATEGORIA']).aggregate(aggregation_functions)
    medioDf = medioDf.append(group_matriculado_estudiando, ignore_index=True)
    
medioDf

In [ ]:
append2Json(outputfile, "medios_clases_distancia", medioDf)

In [ ]:
medioDf_dep = pd.DataFrame()
        
for nivel in niveles:
    matriculado_estudiando = educacion_basica_incompleta['P307'] == 'si'
    matriculado_estudiando_nivel = educacion_basica_incompleta['P308A'] == nivel
    matriculado_estudiando_medio = educacion_basica_incompleta['DEPARTAMENTO'] == 'LIMA'
    data_matriculado_estudiando = educacion_basica_incompleta[matriculado_estudiando & matriculado_estudiando_nivel & matriculado_estudiando_medio]
    data_matriculado_estudiando = data_matriculado_estudiando.reset_index()

    #     group_matriculado_estudiando = data_matriculado_estudiando.groupby(["DEPARTAMENTO", "P308A", "P307A1", "P307A2", "P307A3", "P307A4"]).agg({'FACTOR': 'sum'})
    data_matriculado_estudiando['PERCENTAGE'] = (data_matriculado_estudiando["FACTOR"] / data_matriculado_estudiando['FACTOR'].sum())*100
    data_matriculado_estudiando['PERCENTAGE'] = data_matriculado_estudiando['PERCENTAGE'].round(2)
    data_matriculado_estudiando["TRIMESTRE"] = trimestre
    data_matriculado_estudiando.loc[data_matriculado_estudiando['P307A1'] == "pase", 'P307A1'] = np.nan
    data_matriculado_estudiando.loc[data_matriculado_estudiando['P307A2'] == "pase", 'P307A2'] = np.nan
    data_matriculado_estudiando.loc[data_matriculado_estudiando['P307A3'] == "pase", 'P307A3'] = np.nan
    data_matriculado_estudiando.loc[data_matriculado_estudiando['P307A4'] == "pase", 'P307A4'] = np.nan     
    data_matriculado_estudiando['P307A1'] = data_matriculado_estudiando['P307A1'].replace({"televisión": "TV"})
    data_matriculado_estudiando['P307A2'] = data_matriculado_estudiando['P307A2'].replace({"radio / equipo de sonido": "Radio"})
    data_matriculado_estudiando['P307A3'] = data_matriculado_estudiando['P307A3'].replace({"plataforma virtual / página web": "Web"})
    data_matriculado_estudiando['P307A4'] = data_matriculado_estudiando['P307A4'].replace({"otro": "Otro"})
    data_matriculado_estudiando["CATEGORIA"] = data_matriculado_estudiando[['P307A1', 'P307A2', 'P307A3', 'P307A4']].stack().groupby(level=0).agg(' + '.join)
    data_matriculado_estudiando["CATEGORIA"] = data_matriculado_estudiando['CATEGORIA'].astype(str).replace({"nan": "Otro"})
    aggregation_functions = {'DEPARTAMENTO':'first', 'P308A':'first', 'P307A1':'first', 'P307A2':'first', 'P307A3':'first', 'P307A4':'first', 'FACTOR':'sum','PERCENTAGE': 'sum', 'TRIMESTRE':'first','CATEGORIA': 'first'}
    data_matriculado_estudiando = data_matriculado_estudiando.groupby(data_matriculado_estudiando['CATEGORIA']).aggregate(aggregation_functions)

    medioDf_dep = medioDf_dep.append(data_matriculado_estudiando, ignore_index=True)
    
medioDf_dep

In [ ]:
append2Json(outputfile, "medio_basica_departamento", medioDf_dep)

In [ ]:
#matriculados el año pasado y si aprobó o no
# estudio_ano_pasado = educacion_basica_incompleta.groupby(["P303"]).agg({'FACTOR': 'sum'})

# estudio_ano_pasado.apply(np.round)

In [ ]:
# ano_pasado_estudio = educacion_basica_incompleta['P303'] == 'si'
# data_ano_pasado_estudio = educacion_basica_incompleta[ano_pasado_estudio]
# len(data_ano_pasado_estudio)

In [ ]:
# nivel_resultado = data_ano_pasado_estudio.groupby(["P305"]).agg({'FACTOR': 'sum'})

# nivel_resultado.apply(np.round).reset_index()

In [ ]:
# resultado_desaprobado = data_ano_pasado_estudio['P305'] == 'desaprobado'
# data_resultado_desaprobado = data_ano_pasado_estudio[resultado_desaprobado]
# desaprobados_matriculados = data_resultado_desaprobado.groupby(["P306", "P207"]).agg({'FACTOR': 'sum'})

# desaprobados_matriculados.apply(np.round).reset_index()

In [ ]:
#1 JEFE/JEFA
#2 ESPOSO/ESPOSA 
p203 = data_menor_25.groupby(['P203']).size()
p203

In [ ]:
#nivel de estudios de jefe de hogar por sexo
jefxDf = pd.DataFrame()
jefxs = ["Jefe/Jefa", "Esposo(a)/compañero(a)"]        
for jefx in jefxs:
    jefe_hogar = data_menor_25['P203'] == jefx
    jefe_hogar_nivel = data_menor_25['P301A'].notna()
    data_jefe_hogar = data_menor_25[jefe_hogar & jefe_hogar_nivel]
    hogar_nivel_sexo = data_jefe_hogar.groupby(['P301A', 'P207', 'P203']).agg({'FACTOR': 'sum'})
    hogar_nivel_sexo = hogar_nivel_sexo.reset_index()
    hogar_nivel_sexo["PERCENTAGE"] = (hogar_nivel_sexo["FACTOR"] / hogar_nivel_sexo['FACTOR'].sum())*100
    hogar_nivel_sexo["PERCENTAGE"] = hogar_nivel_sexo["PERCENTAGE"].round(2)
    hogar_nivel_sexo["SEXO"] = np.where(hogar_nivel_sexo["P207"] == "Hombre","Hombres", "Mujeres")
    hogar_nivel_sexo["TRIMESTRE"] = trimestre
    jefxDf = jefxDf.append(hogar_nivel_sexo, ignore_index=True)
    
jefxDf

In [ ]:
append2Json(outputfile, "jefx_genero", jefxDf)

In [ ]:
#Sabe leer y escribir

basica_especial = data_menor_25['P301A'] == 'básica especial'
sin_nivel = data_menor_25['P301A'] == 'sin nivel'
educacion_inicial = data_menor_25['P301A'] == 'educación inicial'
primaria_incompleta = data_menor_25['P301A'] == 'primaria incompleta'

data_leer_escribir = data_menor_25[basica_especial | sin_nivel | educacion_inicial | primaria_incompleta]
len(data_leer_escribir)

In [ ]:
data_leer_escribir['P302'] = data_leer_escribir['P302'].fillna('no respondio')

In [ ]:
# rpta_leer_escribir = data_leer_escribir.groupby(['P302', 'DEPARTAMENTO']).agg({'FACTOR': 'sum'})
# rpta_leer_escribir

In [ ]:
no_sabe_leer = data_leer_escribir['P302'] == 'no'
data_no_sabe_leer = data_leer_escribir[no_sabe_leer]
rpta_leer_escribir = data_no_sabe_leer.groupby(['P300A', 'P208A', 'DOMINIO']).agg({'FACTOR': 'sum'})
rpta_leer_escribir = rpta_leer_escribir.reset_index()
rpta_leer_escribir["TRIMESTRE"] = trimestre
rpta_leer_escribir

In [ ]:
append2Json(outputfile, "tabla_leer_escribir", rpta_leer_escribir)

In [ ]:
# rpta_alfabetizacion = data_leer_escribir.groupby(['P302A']).size()
# rpta_alfabetizacion

In [ ]:
niveles_educ = data_menor_25.groupby(["P301A"]).size()
niveles_educ

In [ ]:
#razones por las q no está matriculado
estudiante_nivel = educacion_basica_incompleta['P301A'].notna()
no_matriculados = educacion_basica_incompleta['P306'] == "no"

data_matriculados_2020 = educacion_basica_incompleta[estudiante_nivel & no_matriculados]
razones_no_matriculados = data_matriculados_2020.groupby(["P313"]).agg({'FACTOR': 'sum'})
razones_no_matriculados = razones_no_matriculados.reset_index()
razones_no_matriculados["PERCENTAGE"] = (razones_no_matriculados["FACTOR"] / razones_no_matriculados['FACTOR'].sum())*100
razones_no_matriculados["PERCENTAGE"] = razones_no_matriculados["PERCENTAGE"].round(2)
razones_no_matriculados["TRIMESTRE"] = trimestre
razones_no_matriculados

In [ ]:
append2Json(outputfile, "razones_no_matriculado", razones_no_matriculados)

In [ ]:
#en que nivel están los matriculados basica y superior
estudiante_nivel = data_menor_25['P301A'].notna()
matriculado_estudiando = data_menor_25['P306'] == 'si'
data_matriculado_estudiando = data_menor_25[matriculado_estudiando & estudiante_nivel]
niveles_estudiando_basica = data_matriculado_estudiando.groupby(["P308A"]).agg({'FACTOR': 'sum'})
niveles_estudiando_basica = niveles_estudiando_basica.reset_index()
niveles_estudiando_basica["PERCENTAGE"] = (niveles_estudiando_basica["FACTOR"] / niveles_estudiando_basica['FACTOR'].sum())*100
niveles_estudiando_basica["TRIMESTRE"] = trimestre
niveles_estudiando_basica

In [ ]:
niveles_estudiando_basica['FACTOR'].sum()

In [ ]:
#append2Json("json/data-2020-01.json", "chart_matriculados_trimestre", niveles_estudiando_basica, True)